In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import tensorflow as tf
import csv
from glob import glob

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_selection import SelectFromModel, VarianceThreshold

import sys;

# Import Data

In [2]:
features_test = pd.read_csv('./dengue_features_test.csv', parse_dates=True, delimiter=',') # , index_col=0
features_train = pd.read_csv('./dengue_features_train.csv', parse_dates=True, delimiter=',')
labels_train = pd.read_csv('./dengue_labels_train.csv', parse_dates=True, delimiter=',')

FileNotFoundError: File b'./dengue_features_test.csv' does not exist

# Drop Certain Features

In [ ]:
#features_train.drop('year', axis=1, inplace=True)
#features_train.drop('weekofyear', axis=1, inplace=True)
features_train.drop('week_start_date', axis=1, inplace=True)
features_train.drop('city', axis=1, inplace=True)

labels_train.drop('year', axis=1, inplace=True)
labels_train.drop('weekofyear', axis=1, inplace=True)
labels_train.drop('city', axis=1, inplace=True)

#features_train.drop('year', axis=1, inplace=True)
#features_train.drop('weekofyear', axis=1, inplace=True)
features_test.drop('week_start_date', axis=1, inplace=True)
features_test.drop('city', axis=1, inplace=True)

# Normalize

In [ ]:
features_train = (features_train -  features_train.mean()) / features_train.std()
features_test = (features_test -  features_test.mean()) / features_test.std()

# Fill in NAs

In [ ]:
features_train.fillna(method='bfill', inplace=True)
features_test.fillna(method='bfill', inplace=True)
labels_train.fillna(method='bfill', inplace=True)

# Split Train/Test

In [ ]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(features_train, 
                                                                  labels_train, test_size=0.3, random_state=101)

# Feature Columns for DNN

In [ ]:
feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(features_train)

# Test Models

In [ ]:
import xgboost as xgb
from sklearn import linear_model
from sklearn import cross_validation
from sklearn import ensemble

#imports
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import load_digits

In [ ]:
lm = linear_model.LinearRegression()

dnn = tf.contrib.learn.DNNRegressor(feature_columns=feature_columns, hidden_units=[20,15], 
                                    activation_fn=tf.nn.relu6) #, tf_random_seed = 101)

xgbr = xgb.XGBRegressor(seed = 101)

gbr = ensemble.GradientBoostingRegressor(random_state = 101) 

rfr = ensemble.RandomForestRegressor(random_state = 101)

etr = ensemble.ExtraTreesRegressor(random_state = 101)

abr = ensemble.AdaBoostRegressor(random_state = 101)

br = ensemble.BaggingRegressor(random_state = 101)

In [ ]:
# Grid search XGBoost

param_grid = { "n_estimators"      : [1, 25],
           "learning_rate"      : [0.01, 0.2],
           "max_depth"         : [5, 20],
           "subsample" : [0.1, 0.99] ,
           "colsample_bytree": [0.01, 0.9]}

# n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(xgbr, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train) # right datasets???
print(grid_search.best_params_)

In [ ]:
# Grid search GradientBoost
 
param_grid = { "n_estimators"      : [275, 325],
           "learning_rate"      : [0.0001, 0.1],
           "max_depth"         : [5, 15],
           "min_samples_leaf": [3,6],
           "subsample" : [0.1, 0.99] ,
           "warm_start": [True, False]}

# max_features=None, alpha=0.9, verbose=0, "min_samples_split": [1,4],

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(gbr, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train)
print(grid_search.best_params_)

In [ ]:
# Grid search RandomForests
 
param_grid = { "n_estimators"      : [275, 325],
           "max_depth"         : [5, 15],
           "min_samples_leaf": [3,6],
           "warm_start": [True, False]}

# max_features=None, alpha=0.9, verbose=0, "min_samples_split": [1,4],

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(rfr, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train)
print(grid_search.best_params_)

In [ ]:
# Grid search ExtraTrees
 
param_grid = { "n_estimators"      : [225, 275],
           "max_depth"         : [5, 15],
           "min_samples_leaf": [3, 6],
           "bootstrap": [True, False],
           "warm_start": [True, False]}           

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(etr, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train)
print(grid_search.best_params_)

In [ ]:
# Grid search AdaBoostingTrees
 
param_grid = { "n_estimators"      : [200, 500],
           "learning_rate"      : [0.00001, 0.1]}           

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(abr, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train)
print(grid_search.best_params_)

In [ ]:
# Grid search BoostingTrees
 
param_grid = { "n_estimators"      : [275, 325],
           "max_samples"         : [5, 15],
           "n_jobs" : [1, 5] ,
           "bootstrap": [True, False],
           "warm_start": [True, False]}           

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(br, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train)
print(grid_search.best_params_)

In [ ]:
#estimator.get_params().keys()

# Final Models

In [ ]:
lm = linear_model.LinearRegression()

dnn = tf.contrib.learn.DNNRegressor(feature_columns=feature_columns, hidden_units=[20,15], 
                                    activation_fn=tf.nn.relu6) #, tf_random_seed = 101)

xgbr = xgb.XGBRegressor(seed = 101, n_estimators = 5, subsample = 0.9, learning_rate = 0.1, colsample_bytree = 0.1, max_depth = 15) 
#{'n_estimators': 5, 'subsample': 0.9, 'learning_rate': 0.1, 'colsample_bytree': 0.1, 'max_depth': 15}

gbr = ensemble.GradientBoostingRegressor(random_state = 101, n_estimators = 300, warm_start = True, max_depth = 10, learning_rate = 0.001, min_samples_leaf = 4, subsample = 0.9) 
#{'n_estimators': 300, 'warm_start': True, 'max_depth': 10, 'learning_rate': 0.001, 'min_samples_leaf': 4, 'subsample': 0.9}

rfr = ensemble.RandomForestRegressor(random_state = 101, n_estimators = 300, warm_start = True, min_samples_leaf = 4, max_depth = 10)
#{'n_estimators': 300, 'warm_start': True, 'min_samples_leaf': 4, 'max_depth': 10}

etr = ensemble.ExtraTreesRegressor(random_state = 101, n_estimators = 250, bootstrap = False, warm_start = True, min_samples_leaf = 4, max_depth = 10)
#{'n_estimators': 250, 'bootstrap': False, 'warm_start': True, 'min_samples_leaf': 4, 'max_depth': 10}

abr = ensemble.AdaBoostRegressor(random_state = 101, n_estimators = 500, learning_rate = 0.00001)
#{'n_estimators': 250, 'learning_rate': 0.001}

br = ensemble.BaggingRegressor(random_state = 101, n_estimators = 300, n_jobs = 1, warm_start = True, max_samples = 10, bootstrap = False)
#{'n_estimators': 300, 'n_jobs': 1, 'warm_start': True, 'max_samples': 10, 'bootstrap': False}

In [ ]:
lm.fit(x_train, y_train)
dnn.fit(x_train, y_train, steps=250)
gbr.fit(x_train, y_train)
rfr.fit(x_train, y_train)
etr.fit(x_train, y_train)
abr.fit(x_train, y_train)
br.fit(x_train, y_train)
xgbr.fit(x_train, y_train);

# MAE

In [ ]:
print('LM Training Error:', metrics.mean_absolute_error(y_train, lm.predict(x_train)))
print('LM Validation Error:', metrics.mean_absolute_error(y_val, lm.predict(x_val)))
diff1 = metrics.mean_absolute_error(y_train, lm.predict(x_train))-metrics.mean_absolute_error(y_val, lm.predict(x_val))
print('Diff:', diff1,'\n')

print('XGB Training Error:', metrics.mean_absolute_error(y_train, xgbr.predict(x_train)))
print('XGB Validation Error:', metrics.mean_absolute_error(y_val, xgbr.predict(x_val)))
diff2 = metrics.mean_absolute_error(y_train, xgbr.predict(x_train))-metrics.mean_absolute_error(y_val, xgbr.predict(x_val))
print('Diff:', diff2,'\n')

print('GBR Training Error:', metrics.mean_absolute_error(y_train, gbr.predict(x_train)))
print('GBR  Validation Error:', metrics.mean_absolute_error(y_val, gbr.predict(x_val)))
diff3 = metrics.mean_absolute_error(y_train, gbr.predict(x_train))-metrics.mean_absolute_error(y_val, gbr.predict(x_val))
print('Diff:', diff3,'\n')

print('RFR Training Error:', metrics.mean_absolute_error(y_train, rfr.predict(x_train)))
print('RFR Validation Error:', metrics.mean_absolute_error(y_val, rfr.predict(x_val)))
diff4 = metrics.mean_absolute_error(y_train, rfr.predict(x_train))-metrics.mean_absolute_error(y_val, rfr.predict(x_val))
print('Diff:', diff4,'\n')

print('ETR Training Error:', metrics.mean_absolute_error(y_train, etr.predict(x_train)))
print('ETR Validation Error:', metrics.mean_absolute_error(y_val, etr.predict(x_val)))
diff5 = metrics.mean_absolute_error(y_train, etr.predict(x_train))-metrics.mean_absolute_error(y_val, etr.predict(x_val))
print('Diff:', diff5,'\n')

print('ABR Training Error:', metrics.mean_absolute_error(y_train, abr.predict(x_train)))
print('ABR Validation Error:', metrics.mean_absolute_error(y_val, abr.predict(x_val)))
diff6 = metrics.mean_absolute_error(y_train, abr.predict(x_train))-metrics.mean_absolute_error(y_val, abr.predict(x_val))
print('Diff:', diff6,'\n')

print('BR Training Error:', metrics.mean_absolute_error(y_train, br.predict(x_train)))
print('BR Validation Error:', metrics.mean_absolute_error(y_val, br.predict(x_val)))
diff7 = metrics.mean_absolute_error(y_train, br.predict(x_train))-metrics.mean_absolute_error(y_val, br.predict(x_val))
print('Diff:', diff7,'\n')

print('DNN Training Error:', metrics.mean_absolute_error(y_train, list(dnn.predict(x_train, as_iterable=True))))
#print('DNN Validation Error:', metrics.mean_absolute_error(y_val, list(dnn.predict(x_val, as_iterable=True)),'\n'));

# Fit Models

In [ ]:
lm.fit(features_train, labels_train)

In [ ]:
dnn.fit(features_train, labels_train, steps = 250)

In [ ]:
xgbr.fit(features_train, labels_train)

In [ ]:
gbr.fit(features_train, labels_train)

In [ ]:
rfr.fit(features_train, labels_train)

In [ ]:
etr.fit(features_train, labels_train)

In [ ]:
abr.fit(features_train, labels_train)

In [ ]:
br.fit(features_train, labels_train)

In [ ]:
final_lm = lm.predict(features_train)
final_dnn = dnn.predict(features_train, as_iterable=True)
final_xgbr = xgbr.predict(features_train)
final_gbr = gbr.predict(features_train)
final_rfr = rfr.predict(features_train)
final_etr = etr.predict(features_train)
final_abr = abr.predict(features_train)
final_br = br.predict(features_train)

In [ ]:
final_dnn = np.asarray(final_dnn)

In [ ]:
final_dnn

In [ ]:
final_lm[final_lm < 0] = 0
final_lm = final_lm.astype(dtype='int64')

#final_dnn[final_dnn < 0] = 0
#final_dnn = final_dnn.astype(dtype='int64')

final_xgbr[final_xgbr < 0] = 0
final_xgbr = final_xgbr.astype(dtype='int64')

final_gbr[final_gbr < 0] = 0
final_gbr = final_gbr.astype(dtype='int64')

final_rfr[final_rfr < 0] = 0
final_rfr = final_rfr.astype(dtype='int64')

final_etr[final_etr < 0] = 0
final_etr = final_etr.astype(dtype='int64')

final_abr[final_abr < 0] = 0
final_abr = final_abr.astype(dtype='int64')

final_br[final_br < 0] = 0
final_br = final_br.astype(dtype='int64')

In [ ]:
features_train.shape

# Final Ensemble Model

In [ ]:
FINAL = np.hstack((final_lm, np.reshape(final_xgbr, (1456,1)), np.reshape(final_gbr, (1456,1)), 
                   np.reshape(final_rfr, (1456,1)), np.reshape(final_etr, (1456,1)), 
                   np.reshape(final_abr, (1456,1)), np.reshape(final_br, (1456,1)))) 

'''features_train)) #, final_dnn'''

In [ ]:
xgbr2 = xgb.XGBRegressor(seed = 100)

In [ ]:
# Grid search XGBoost

param_grid = { "n_estimators"      : [1, 25],
           "learning_rate"      : [0.01, 0.2],
           "max_depth"         : [5, 20],
           "subsample" : [0.1, 0.99] ,
           "colsample_bytree": [0.01, 0.9]}

# n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5

# INSERT YOUR MODEL HERE WITH NO PARAMETERS DEFINED EXCEPT RANDOM SEED
grid_search = GridSearchCV(xgbr2, param_grid, n_jobs=-1, cv=2)
grid_search.fit(features_train, labels_train) # right datasets???
print(grid_search.best_params_)

In [ ]:
xgbr2 = xgb.XGBRegressor(seed = 100, n_estimators = 5, subsample = 0.9, learning_rate = 0.1, colsample_bytree = 0.5, max_depth = 15) 
#{'n_estimators': 5, 'subsample': 0.9, 'learning_rate': 0.1, 'colsample_bytree': 0.1, 'max_depth': 15}

In [ ]:
xgbr2.fit(FINAL, labels_train)

In [ ]:
final_xgbr2 = xgbr2.predict(features_test)

In [ ]:
final_xgbr2[final_xgbr2 < 0] = 0
final_xgbr2 = final_xgbr2.astype(dtype='int64')

In [ ]:
np.savetxt("final_ensemble.csv", FINAL, delimiter=",")

In [ ]:
Stacker(xgbr)

In [ ]:
g = ensemble.GradientBoostingRegressor()
l = linear_model.LinearRegression()

In [ ]:
x[: 80, 0] = Stacker(g).fit_transform(x0[: 80, :], y[: 80])[:, 0]
x[: 80, 1] = Stacker(l).fit_transform(x1[: 80, :], y[: 80])[:, 0]

u = linear_model.LinearRegression().fit(x[: 80, :], y[: 80])

x[80: , 0] = Stacker(g).fit(x0[: 80, :], y[: 80]).transform(x0[80:, :])
x[80: , 1] = Stacker(l).fit(x1[: 80, :], y[: 80]).transform(x1[80:, :])

metrics.r2_score(
        y[80: ],
        u.predict(x[80:, :]))
    0.992196564279